In [1]:
import tensorflow as tf

In [2]:
class NaiveDense:
  # Intialize variables
  def __init__(self, input_size, output_size, activation):
    # Create activation object linked to activation input
    self.activation = activation

    '''
    Create the shape of the weights matrix where input_size corresponds to
    the number of features and output_size to the number of neurons.
    This shape enables matrix multiplication between the inputs and weights.

    Initialize the weights using a uniform distribution, which samples values
    evenly between 0 and 0.1. This keeps initial weights small, helping to
    avoid exploding outputs or gradients during training.

    Wrap the initialized weights in a TensorFlow Variable so they can be updated
    during training via backpropagation.
    '''
    w_shape        = (input_size, output_size)
    w_intial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W         = tf.Variable(w_intial_value)
Iterate batch sizes sequentially
    '''
    Create the shape of the bias vector using output_size, as each neuron
    gets a single bias value.

    Initialize all biases to zero and wrap them in a TensorFlow Variable
    so they can also be updated during training.
    '''
    b_shape        = (output_size,)
    b_intial_value = tf.zeros(b_shape)
    self.b         = tf.Variable(b_intial_value)

  # Define how the layer is called on inputs — performs the forward pass
  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  # Provide a convenient way to access the layer's weights and biases
  # \@property allows the functions to act similar to a variable
  @property
  def weights(self):
    return [self.W, self.b]


In [4]:
class NaiveSequential:
  # Intialize thw self.layers object for further automation
  def __init__(self, layers):
    self.layers = layers

  '''
  Defines how the model handles input data.

  When the model is called with input data, it passes the data sequentially
  through each NaiveDense layer by invoking that layer’s __call__ method.

  Each layer takes in the input, performs its own weight and bias
  computation (via NaiveDense.__call__), and passes the result to the next layer.

  The final output after all layers is returned.
  '''
  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  '''
  Collects and returns all the weights and biases from each NaiveDense layer.

  Each NaiveDense instance has a `weights` property that returns a list
  containing its weight and bias tensors. This method gathers all such
  lists into a single flat list across all layers in the model.
  '''
  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [7]:
# Call the sequential models this will create all the weights for all the layers.
model = NaiveSequential([
    NaiveDense(input_size = 28 * 28, output_size = 512, activation = tf.nn.relu),
    NaiveDense(input_size = 512, output_size = 10, activation = tf.nn.softmax)
])

assert len(model.weights) == 4

In [ ]:
import math

class BatchGenerator:
  # Intialize relevant objects for calculating batch sizes
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index      = 0
    self.images     = images
    self.labels     = labels
    self.batch_size = batch_size
    self.batch_num  = math.ceil(len(images) / self.batch_size)

  # Return the next batch of images and labels
  def next(self):
    images = self.images[self.index : self.index+self.batch_size]
    labels = self.labels[self.index : self.index+self.batch_size]
    self.index += self.batch_size
    return images, labels